In [1]:
import re
from thesisutils import utils
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [2]:
import spacy
import os
nlp = spacy.load("en_core_web_lg")
# nlp = spacy.load("en_core_web_md")

In [3]:
def get_doc_nerchars(row):
    """Creates start and end_char columns for ner df based on 
    token indexes of doc of current row of maindf. 
    """
    # print(row[pub.uidcol])
    try:
        doc = nlp(row[pub.textcol], disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"])
    except ValueError as ve:
        # print(row[pub.uidcol])
        # print(ve)
        return None

    mask = nerdf[pub.uidcol].eq(row[pub.uidcol])
    try:
        nerdf.loc[mask, "start_char"], nerdf.loc[mask, "end_char"] = zip(*nerdf[mask].apply(nerchar, doc=doc, axis=1))
    except ValueError as ve:
        # print(row[pub.uidcol])
        # print(ve)
        return None
        
    
def nerchar(nrow, doc):
    """Gets the start/end_chars of tokens in ner."""
    # print(nrow.ner_index)
    ent = doc[nrow["start"]:nrow["end"]]
    return ent.start_char, ent.end_char

In [90]:
pub = utils.publications['nyt']

maindf = utils.read_df_s3(f"{pub.name}/{pub.name}_full.csv")
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_full.csv")

nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)

path = f"../../data/{pub.name}/ner/ner_full2.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 

In [94]:
blocksize = 200
for i in tqdm(range(0, len(maindf), blocksize)):
    idx = maindf.iloc[i:i+blocksize][pub.uidcol]
    maindf.iloc[i:i+blocksize].apply(get_doc_nerchars, axis=1)
    nerdf[nerdf[pub.uidcol].isin(idx)].to_csv(f"../../data/{pub.name}/ner/ner_full{i}.csv")

100%|██████████| 1/1 [00:03<00:00,  3.73s/it]


In [100]:
print(nerdf.start_char.isna().value_counts())
nerdf.start_char = nerdf.start_char.astype(pd.Int64Dtype())
nerdf.end_char = nerdf.end_char.astype(pd.Int64Dtype())
nerdf.to_csv(f"../../data/{pub.name}/ner/ner_full2.csv")
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full2.csv")


In [103]:
pub = utils.publications['globaltimes']


In [106]:
def run(pub, start=0):
    # pub = utils.publications['nyt']

    maindf = utils.read_df_s3(f"{pub.name}/{pub.name}_full.csv")
    nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_full.csv")

    nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
    maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
    nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)

    path = f"../../data/{pub.name}/ner/ner_full2.csv"
    os.makedirs(os.path.dirname(path), exist_ok=True) 
    blocksize = 200
    for i in tqdm(range(start, len(maindf), blocksize)):
        idx = maindf.iloc[i:i+blocksize][pub.uidcol]
        maindf.iloc[i:i+blocksize].apply(get_doc_nerchars, axis=1)
        nerdf[nerdf[pub.uidcol].isin(idx)].to_csv(f"../../data/{pub.name}/ner/ner_full{i}.csv")
    print(nerdf.start_char.isna().value_counts())
    nerdf.start_char = nerdf.start_char.astype(pd.Int64Dtype())
    nerdf.end_char = nerdf.end_char.astype(pd.Int64Dtype())
    nerdf.to_csv(f"../../data/{pub.name}/ner/ner_full2.csv")
    utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full2.csv")

In [108]:
print(pub)
maindf = utils.read_df_s3(f"{pub.name}/{pub.name}_full.csv")
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_full.csv")

globaltimes


/home/ec2-user/SageMaker/.persisted_conda/nlp/lib/python3.10/site-packages/thesisutils/utils.py:124: DtypeWarning: Columns (13,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(csv_string))


In [109]:
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)

path = f"../../data/{pub.name}/ner/ner_full2.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 

In [111]:
start =0
blocksize = 200
for i in tqdm(range(start, len(maindf), blocksize)):
    idx = maindf.iloc[i:i+blocksize][pub.uidcol]
    maindf.iloc[i:i+blocksize].apply(get_doc_nerchars, axis=1)
    nerdf[nerdf[pub.uidcol].isin(idx)].to_csv(f"../../data/{pub.name}/ner/ner_full{i}.csv")

 17%|█▋        | 18/108 [03:14<16:10, 10.79s/it]

790013
too many values to unpack (expected 2)


 19%|█▊        | 20/108 [03:35<15:36, 10.64s/it]

803507
too many values to unpack (expected 2)


 61%|██████    | 66/108 [11:56<07:26, 10.62s/it]

...

 82%|████████▏ | 89/108 [16:29<04:51, 15.34s/it]

1189896
too many values to unpack (expected 2)


 83%|████████▎ | 90/108 [16:44<04:32, 15.13s/it]

1193176
[E1041] Expected a string, Doc, or bytes as input, but got: <class 'float'>


100%|██████████| 108/108 [20:00<00:00, 11.11s/it]


In [112]:
pub = utils.publications['hkfp']

print(pub)
maindf = utils.read_df_s3(f"{pub.name}/{pub.name}_full.csv")
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_full.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)

path = f"../../data/{pub.name}/ner/ner_full2.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
start =0
blocksize = 200
for i in tqdm(range(start, len(maindf), blocksize)):
    idx = maindf.iloc[i:i+blocksize][pub.uidcol]
    maindf.iloc[i:i+blocksize].apply(get_doc_nerchars, axis=1)
    nerdf[nerdf[pub.uidcol].isin(idx)].to_csv(f"../../data/{pub.name}/ner/ner_full{i}.csv")

hkfp


  0%|          | 0/102 [00:00<?, ?it/s]

post-18527
too many values to unpack (expected 2)
post-18531
too many values to unpack (expected 2)
post-18623
too many values to unpack (expected 2)
post-15387
too many values to unpack (expected 2)


 10%|▉         | 10/102 [01:23<12:48,  8.35s/it]

post-154501
too many values to unpack (expected 2)


...

 68%|██████▊   | 69/102 [09:45<04:31,  8.23s/it]

post-55199
too many values to unpack (expected 2)


 81%|████████▏ | 83/102 [11:40<02:41,  8.48s/it]

post-88332
too many values to unpack (expected 2)


100%|██████████| 102/102 [14:19<00:00,  8.43s/it]


In [13]:
pub = utils.publications['globaltimes']
maindf = utils.read_df_s3(f"{pub.name}/{pub.name}_full.csv") 
ndf = pd.concat(pd.read_csv(f"../../data/{pub.name}/ner/ner_full{i}.csv") for i in range(0, len(maindf), blocksize))


/home/ec2-user/SageMaker/.persisted_conda/nlp/lib/python3.10/site-packages/thesisutils/utils.py:124: DtypeWarning: Columns (13,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(csv_string))


In [14]:
ndf

,Unnamed: 0,Unnamed: 0.1,ner_index,entity,label_,start,end,Art_id,publication,start_char,end_char
0,0,0,0,Liu Rui/GT,PERSON,2,6,1241537,globaltimes,14.0,24.0
1,1,1,1,US,GPE,8,9,1241537,globaltimes,30.0,32.0
2,2,2,2,US,GPE,29,30,1241537,globaltimes,122.0,124.0
3,3,3,3,Joe Biden,PERSON,31,33,1241537,globaltimes,135.0,144.0
4,4,4,4,Yury Tavrovsky,PERSON,47,49,1241537,globaltimes,211.0,225.0
...,...,...,...,...,...,...,...,...,...,...,...
2246,910398,1060184,1060212,the White House Internship Office,ORG,524,529,625615,globaltimes,2689.0,2722.0
2247,910399,1060185,1060213,Binjiang Avenue,FAC,13,15,625830,globaltimes,66.0,81.0
2248,910400,1060186,1060214,Hong Kong,GPE,23,25,625830,globaltimes,128.0,137.0
2249,910401,1060187,1060215,Bund,FAC,28,29,625830,globaltimes,152.0,156.0


In [15]:
path = f"../../data/{pub.name}/ner/ner_full2.csv"
ndf.start_char = ndf.start_char.astype(pd.Int64Dtype())
ndf.end_char = ndf.end_char.astype(pd.Int64Dtype())
ndf.to_csv(path)
utils.df_to_s3(ndf, f"{pub.name}/ner/ner_full2.csv")

In [7]:
blocksize=200

In [5]:
pub = utils.publications['scmp']


In [14]:
pub.name

'scmp'

In [18]:
# def scmprun(year):
year = 2011
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

  2%|▏         | 511/20516 [00:21<13:58, 23.86it/s]

722527
too many values to unpack (expected 2)


  5%|▍         | 976/20516 [00:40<13:04, 24.90it/s]

723024
too many values to unpack (expected 2)


  6%|▌         | 1195/20516 [00:49<12:43, 25.31it/s]

...

 94%|█████████▍| 19288/20516 [12:53<00:48, 25.23it/s]

743052
too many values to unpack (expected 2)
743053
too many values to unpack (expected 2)


 95%|█████████▍| 19480/20516 [13:00<00:40, 25.63it/s]

743248
too many values to unpack (expected 2)


100%|██████████| 20516/20516 [13:41<00:00, 24.97it/s]


In [19]:
year = 2012
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

  2%|▏         | 383/18033 [00:14<10:53, 27.03it/s]

757879
too many values to unpack (expected 2)


  9%|▉         | 1635/18033 [01:00<10:04, 27.13it/s]

759222
too many values to unpack (expected 2)


 11%|█         | 1938/18033 [01:11<09:42, 27.62it/s]

...

 97%|█████████▋| 17552/18033 [10:39<00:17, 27.51it/s]

777157
too many values to unpack (expected 2)


100%|█████████▉| 18014/18033 [10:56<00:00, 27.70it/s]

777651
too many values to unpack (expected 2)


100%|██████████| 18033/18033 [10:57<00:00, 27.43it/s]


In [21]:
year = 2013
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

100%|██████████| 22948/22948 [17:10<00:00, 22.26it/s]


In [22]:
year = 2014
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

100%|██████████| 26399/26399 [15:21<00:00, 28.64it/s] 


In [23]:
year = 2015
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

100%|██████████| 24084/24084 [18:08<00:00, 22.13it/s]


In [24]:
year = 2016
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

100%|██████████| 19616/19616 [12:27<00:00, 26.26it/s]


In [25]:
year = 2017
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

100%|██████████| 24901/24901 [24:02<00:00, 17.26it/s]


In [26]:
year = 2018
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

100%|██████████| 24980/24980 [26:04<00:00, 15.96it/s]


In [27]:
year = 2019
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

100%|██████████| 25913/25913 [31:07<00:00, 13.88it/s]


In [6]:
year = 2020
nerdf = utils.read_df_s3(f"{pub.name}/ner/ner_{year}.csv")
maindf = utils.read_df_s3(f"{pub.name}/{year}.csv")
nerdf[pub.uidcol] = nerdf[pub.uidcol].astype(str)
maindf[pub.uidcol] = maindf[pub.uidcol].astype(str)
nerdf = nerdf.rename({"Unnamed: 0": "ner_index"}, axis=1)
maindf.progress_apply(get_doc_nerchars, axis=1)
path = f"../../data/{pub.name}/ner/ner_full_{year}.csv"
os.makedirs(os.path.dirname(path), exist_ok=True) 
nerdf.to_csv(path)
utils.df_to_s3(nerdf, f"{pub.name}/ner/ner_full_{year}.csv")

100%|██████████| 28297/28297 [34:55<00:00, 13.50it/s] 


In [4]:
start =0
blocksize = 200
for i in tqdm(range(start, len(maindf), blocksize)):
    idx = maindf.iloc[i:i+blocksize][pub.uidcol]
    maindf.iloc[i:i+blocksize].apply(get_doc_nerchars, axis=1)
    nerdf[nerdf[pub.uidcol].isin(idx)].to_csv(f"../../data/{pub.name}/ner/ner_full{i}.csv")

NameError: name 'pub' is not defined